In [1]:
from faker import Faker

In [2]:
fake = Faker()

fake.name()

'Erin Murillo'

In [3]:
fake.word()

'range'

In [2]:
import yaml
import os

pwd = os.getcwd()
print(pwd)

/home/phinguyen/ETL_Pipeline_with_Spark_01/notebooks


In [3]:
parent_directory = os.path.dirname(pwd)
parent_directory

'/home/phinguyen/ETL_Pipeline_with_Spark_01'

In [4]:
sample_data_dir = os.path.join(parent_directory, 'sample_data')
sample_data_dir

'/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data'

In [5]:
from pathlib import Path

sample_data_dir = Path(sample_data_dir)
sample_data_dir

PosixPath('/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data')

In [7]:
yaml_file_path = sample_data_dir / 'DimProducts.yaml'
yaml_file_path

PosixPath('/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data/DimProducts.yaml')

In [8]:
with open(yaml_file_path, 'r') as yaml_file:
            yaml_data = yaml.safe_load(yaml_file)
            columns = ', '.join([f"{col['name']} {col['type']}" for col in yaml_data['columns']])
            create_table_query = f"CREATE TABLE IF NOT EXISTS DimProducts ({columns})"
            print(create_table_query)

CREATE TABLE IF NOT EXISTS DimProducts (ProductKey INT, ProductAlternateKey VARCHAR(255), ProductCategoryKeyEnglishProductName VARCHAR(255), Color VARCHAR(255), StandardCost FLOAT, ListPrice FLOAT)


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

user=os.getenv('MYSQL_USER')
print(user)

admin


In [4]:
import yaml

yaml_file_path = "/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data/DimProducts.yaml"
table_name = "DimProducts"
with open(yaml_file_path, 'r') as yaml_file:
    yaml_data = yaml.safe_load(yaml_file)
    print(yaml_data['columns'])
    # Filter out non-dictionary elements
    valid_columns = [col for col in yaml_data['columns'] if isinstance(col, dict)]
    
    # Construct the table columns
    columns = ', '.join([f"{col['name']} {col['type']}" for col in valid_columns])
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})"
print(create_table_query)

[{'name': 'ProductKey', 'type': 'INT'}, {'name': 'ProductAlternateKey', 'type': 'VARCHAR(255)'}, {'name': 'ProductCategoryKeyEnglishProductName', 'type': 'VARCHAR(255)'}, {'name': 'Color', 'type': 'VARCHAR(255)'}, {'name': 'StandardCost', 'type': 'FLOAT'}, {'name': 'ListPrice', 'type': 'FLOAT'}]
CREATE TABLE IF NOT EXISTS DimProducts (ProductKey INT, ProductAlternateKey VARCHAR(255), ProductCategoryKeyEnglishProductName VARCHAR(255), Color VARCHAR(255), StandardCost FLOAT, ListPrice FLOAT)


In [3]:
from src.generator.schema import load_schema

schema = load_schema("/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data/DimProducts.yaml")
schema

ImportError: attempted relative import with no known parent package

In [27]:
import pandas as pd

df = pd.read_csv("/home/phinguyen/ETL_Pipeline_with_Spark_01/sample_data/DimProducts.csv", 
                 na_values=['NA', 'NULL'],
                 quotechar='"')
df.head()

,ProductKey,ProductAlternateKey,ProductCategoryKeyEnglishProductName,Color,StandardCost,ListPrice
1,AR-5381,NaN,Adjustable Race,NaN,NaN,NaN
2,BA-8327,NaN,Bearing Ball,NaN,NaN,NaN
3,BE-2349,NaN,BB Ball Bearing,NaN,NaN,NaN
4,BE-2908,NaN,Headset Ball Bearings,NaN,NaN,NaN
5,BL-2036,NaN,Blade,NaN,NaN,NaN


In [28]:
df[df['ProductKey'] == 'BK-M38S-38']

,ProductKey,ProductAlternateKey,ProductCategoryKeyEnglishProductName,Color,StandardCost,ListPrice
587,BK-M38S-38,1.0,"Mountain-400-W Silver, 38",Silver,419.7784,769.49


In [21]:
import numpy as np

df.replace({np.nan: None}, inplace=True)
df.head()

,ProductKey,ProductAlternateKey,ProductCategoryKeyEnglishProductName,Color,StandardCost,ListPrice
1,AR-5381,None,Adjustable Race,None,None,None
2,BA-8327,None,Bearing Ball,None,None,None
3,BE-2349,None,BB Ball Bearing,None,None,None
4,BE-2908,None,Headset Ball Bearings,None,None,None
5,BL-2036,None,Blade,None,None,None


In [26]:
# Get the column names from the DataFrame
columns = df.columns.tolist()

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Construct the SQL query
    values = ', '.join(['NULL' if v is None else f"'{v}'" for v in row])
    columns_str = ', '.join(columns)
    sql_query = f"INSERT INTO your_table_name ({columns_str}) VALUES ({values})"
    print(sql_query)
    # Print the SQL query with the actual values
    print(sql_query % tuple(row))
    break

INSERT INTO your_table_name (ProductKey, ProductAlternateKey, ProductCategoryKeyEnglishProductName, Color, StandardCost, ListPrice) VALUES ('AR-5381', NULL, 'Adjustable Race', NULL, NULL, NULL)


TypeError: not all arguments converted during string formatting

In [10]:
df[:1]

,ProductKey,ProductAlternateKey,ProductCategoryKeyEnglishProductName,Color,StandardCost,ListPrice
1,AR-5381,NaN,Adjustable Race,NaN,NaN,NaN


In [6]:
df.columns

Index(['ProductKey', 'ProductAlternateKey',
       'ProductCategoryKeyEnglishProductName', 'Color', 'StandardCost',
       'ListPrice'],
      dtype='object')

In [12]:
df = df.where(pd.notna(df), None)

In [15]:
df = df.applymap(lambda x: None if pd.isna(x) else x)

/tmp/ipykernel_82868/4271472690.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if pd.isna(x) else x)


In [19]:
for row in df.itertuples(index=False, name=None):
    print(row)
    for element in row:
        print(element)
        print(type(element))
    break

('AR-5381', nan, 'Adjustable Race', nan, nan, nan)
AR-5381
<class 'str'>
nan
<class 'float'>
Adjustable Race
<class 'str'>
nan
<class 'float'>
nan
<class 'float'>
nan
<class 'float'>


In [ ]:
columns = ['ProductKey', 'ProductAlternateKey',
       'ProductCategoryKeyEnglishProductName', 'Color', 'StandardCost',
       'ListPrice']

insert_query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

for row in df.itertuples(index=False, name=None):
    print(row)
    break

In [5]:
from datetime import datetime, timedelta

def main(n_each_day: int, start_date: str, end_date: str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    number_of_days = (end_date - start_date).days + 1  # Include end_date in the range
    
    # Generate list of dates
    date_list = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(number_of_days)]
    
    print(date_list)
    return date_list

main(10, "2022-10-15", "2022-10-20")

['2022-10-15', '2022-10-16', '2022-10-17', '2022-10-18', '2022-10-19', '2022-10-20']


['2022-10-15',
 '2022-10-16',
 '2022-10-17',
 '2022-10-18',
 '2022-10-19',
 '2022-10-20']

In [1]:
customers_data = []
customer_data_1 = [1, 2, 3]
customer_data_2 = [4, 5, 6]
customers_data.extend(customer_data_1)
customers_data.extend(customer_data_2)
customers_data

[1, 2, 3, 4, 5, 6]